In [1]:
from PIL import Image, ImageDraw
from math import floor, sqrt
import numpy as np
import scipy.stats as stats

In [2]:
M = 3
N = 5

In [3]:
def change_size(image, coefficient, format_ = 'RGBA'):
    width, height = image.size
    new_width = floor(width * coefficient)
    new_height = floor(height * coefficient)
    newImage = Image.new(format_, (new_width, new_height)) 
    for x in range(new_width):
        for y in range(new_height):
            newImage.putpixel((x, y), image.getpixel((floor(x / coefficient), floor(y / coefficient))))
    return newImage

In [4]:
def change_to_semitone(image):
    width, height = image.size
    newImage = Image.new('P', (width, height))
    for x in range(width):
        for y in range(height):
            color = image.getpixel((x,y))
            bright = floor(0.3 * color[0] + 0.59 * color[1] +  0.11 *color[2])
            newImage.putpixel((x, y), bright)
    return newImage

In [5]:
def get_integral_image(image):
    width, height = image.size
    integral_image = np.zeros((height, width))
    for i in range(height):
        sum_ = 0
        for j in range(width):
            sum_ += image.getpixel((j, i))
            if (i == 0):
                integral_image[i][j] = sum_
            else:
                integral_image[i][j] = integral_image[i-1][j] + sum_
    return integral_image

In [6]:
def get_matrices(matrix_image, integral_image):
    height, width = matrix_image.shape
    M = np.zeros((height, width))
    stdev = np.zeros((height, width))
    r = 15
    s2 = r // 2
    for i in range(height):
        for j in range(width):
            x1 = j - s2
            x2 = j + s2
            y1 = i - s2
            y2 = i + s2
        
            if (x1 < 0):
                x1 = 0
            
            if (x2 >= width):
                x2 = width - 1
            
            if (y1 < 0):
                y1 = 0
            
            if (y2 >= height):
                y2 = height - 1
            
            count = (x2 - x1) * (y2 - y1)
            sum_ = integral_image[y2][x2] - integral_image[y1][x2] - integral_image[y2][x1] + integral_image[y1][x1]
            M[i][j] = floor(sum_ / count)
            aperture = matrix_image[y1:y2+1,x1:x2+1]
            stdev[i][j] = floor(sqrt(stats.describe(aperture.flatten())[3]))
    
    return M, stdev

In [7]:
def change_to_binary(image, N):
    new_image = change_size(image, 1/N, 'P')
    matrix_image = np.array(new_image, dtype = 'uint32')
    width, height = new_image.size
    integral_image = get_integral_image(new_image)
    M, stdev = get_matrices(matrix_image, integral_image)         
    bottom_bound, upper_bound = stats.describe(M.flatten())[1]    
    binary_image = Image.new('1', (width, height))    
    for i in range(height):
        for j in range(width):
            if (matrix_image[i][j] < bottom_bound):
                binary_image.putpixel((j, i), 0)
            else:
                if (matrix_image[i][j] > upper_bound):
                    binary_image.putpixel((j, i), 1)
                else:
                    t = M[i][j] - 0.2 * stdev[i][j]
                    if (matrix_image[i][j] > t):
                        binary_image.putpixel((j, i), 1)
                    else:
                        binary_image.putpixel((j, i), 0)
    
    return binary_image

1) Растяжение изображения в M раз

In [8]:
WallE_image = Image.open("pictures/WallE/WallE.bmp")

In [9]:
new_WallE_image = change_size(WallE_image, M)

In [10]:
new_WallE_image.save("pictures/WallE/newWallE.bmp")

2) Сжатие изображения в N раз


In [11]:
Elsa_image = Image.open("pictures/Elsa/Elsa.bmp")

In [12]:
new_Elsa_image = change_size(Elsa_image, 1/N)

In [13]:
new_Elsa_image.save("pictures/Elsa/newElsa.bmp")

3) Передискретизация изображения в K=M/N раз путём растяжения и
последующего сжатия

In [14]:
TLOFTR_image = Image.open("pictures/TheLordOfTheRings/TheLordOfTheRings.bmp")

In [15]:
new_TLOFTR_image = change_size(TLOFTR_image, M)

In [16]:
new_TLOFTR_image = change_size(new_TLOFTR_image, 1/N)

In [17]:
new_TLOFTR_image.save("pictures/TheLordOfTheRings/newTheLordOfTheRings.bmp")

4) Передискретизация изображения в K раз за один проход.

In [18]:
Vinni_Puh_image = Image.open("pictures/VinniPuh/VinniPuh.bmp")

In [19]:
new_Vinni_Puh_image = change_size(Vinni_Puh_image, M/N)

In [20]:
new_Vinni_Puh_image.save("pictures/VinniPuh/new_VinniPuh.bmp")

Приведение полноцветного изображения к полутоновому.

In [21]:
Disgusting_Men_image = Image.open("pictures/DisgustingMen/DisgustingMen.bmp")

In [22]:
new_Disgusting_Men_image = change_to_semitone(Disgusting_Men_image)

In [23]:
new_Disgusting_Men_image.save("pictures/DisgustingMen/newDisgustingMen.bmp")

In [24]:
text_image = Image.open("pictures/text/text.bmp")

In [25]:
new_text_image = change_to_semitone(text_image)

In [26]:
new_text_image.save("pictures/text/new_text.bmp")

Приведение полутонового изображения к монохромному методом пороговой
обработки. Алгоритм адаптивной бинаризации Ниблэка.

In [27]:
binary_image = change_to_binary(new_text_image, 4)

In [28]:
binary_image.show()

In [30]:
binary_image.save("pictures/text/binary_text_image.bmp")